In [ ]:
import os
import pandas as pd

Paths are initialized

In [ ]:
#wikiart (large dataset) will be added later

kaggle_train_path = "../data/train"
kaggle_test_path = "../data/test"

features_path = "../features"
painters_path = "../data/artists.csv"

kaggle_painter_names = os.listdir(kaggle_train_path)
painters = pd.read_csv(painters_path)
painters = painters["name"]